# Smoothing the curve 📈

## From dictionaries to tables

Let's recap the concept of dictionary and the use of `group`:

In [1]:
group `a`b`a`c`d

a| 0 2
b| ,1
c| ,3
d| ,4


Dictionaries are just mappings between keys and values. We can easily create a dictionary by means of `!`, taking two lists as arguments.

In [2]:
`a`b!1 2

a| 1
b| 2


<div class="alert alert-warning">
Take care when creating dictionaries with just one key and one associated value. Note that the syntax `1!2` is wrong. Instead, since `!` expects lists as left and right arguments, you need to lift the values manually, as in: `enlist[1]!enlist 2`.
</div>

When a dictionary contains lists of equal length, and the keys are symbols, we create a *column dictionary*.

In [3]:
show d:`a`b!(til 4;til 4)

a| 0 1 2 3
b| 0 1 2 3


We can convert a column dictionary into a table by applying `flip`. This turns the dictionary into a table where keys become column names and values become the rows.

In [4]:
flip d

a b
---
0 0
1 1
2 2
3 3


<div class="alert alert-warning">
One of the keys of why kdb is so efficient is that kdb database is column oriented rather than row-oriented, as in traditional relational databases like SQL. This means data is stored by columns, not rows.
</div>

Going back to our problem, we need to gen a price table.
What do we need to gen one example of our price table?
- Temporal column
- Sym column
- Price column

To build the temporal column whe have the built-in `hour` type, that simplifies working with time values, allowing you to perform operations directly on hours.

In [28]:
9#09:00+til 3

09:00 09:01 09:02 09:00 09:01 09:02 09:00 09:01 09:02


But we need something more, we need to sort ascendelly the hours, so we are going to use `asc` keyword to handle that

In [29]:
show ts:asc 9#09:00+til 3

`s#09:00 09:00 09:00 09:01 09:01 09:01 09:02 09:02 09:02


<div class="alert alert-warning">
Look a the `s#` at the beginning of the output. It means that, that list has the *sorted* atribbute, meaning the data is recognized by q as being in sorted order, which allows for faster lookups and optimizations during queries
</div>

Similarly, we can create a list of symbols for the assets

In [6]:
show syms:9#`AAPL`AMZN`GOOG

`AAPL`AMZN`GOOG`AAPL`AMZN`GOOG`AAPL`AMZN`GOOG


Finally, we are going to need to generate random data, lets use `roll` `?`, for example to roll a dice 10 times, and set a seed with `\S` to ensure the same output each time

In [32]:
\S 7
10?6

3 3 0 0 1 0 0 5 2 4


Left number is the number of times and right number is the limit number, so let's first generate starting prices 

In [33]:
3?100f 

17.57755 93.62212 15.20677


And then we generate randomly the price jumps between each instant of time. In order to sum the jumps to the starting points, we need to reshape the starting points, so both lists conform 

In [34]:
\S 7
show p: (9?0.05) + 9#3?100f 

96.26213 78.44964 75.3932 96.27035 78.42006 75.3768 96.27845 78.42342 75.37487


Now, we have the ingredients to create our column dictionary ...

In [11]:
show d: `t`sym`price!(ts;syms;p)

t    | 09:00    09:00    09:00   09:01    09:01    09:01   09:02    09:02    ..
sym  | AAPL     AMZN     GOOG    AAPL     AMZN     GOOG    AAPL     AMZN     ..
price| 96.26213 78.44964 75.3932 96.27035 78.42006 75.3768 96.27845 78.42342 ..


... and our table will be ready just flipping the dictionary

In [12]:
show t1: flip d

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:00 AMZN 78.44964
09:00 GOOG 75.3932 
09:01 AAPL 96.27035
09:01 AMZN 78.42006
09:01 GOOG 75.3768 
09:02 AAPL 96.27845
09:02 AMZN 78.42342
09:02 GOOG 75.37487


Finally, there's another syntax to create a dictionary

In [35]:
([]t:ts;sym:syms;price:p)

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:00 AMZN 78.44964
09:00 GOOG 75.3932 
09:01 AAPL 96.27035
09:01 AMZN 78.42006
09:01 GOOG 75.3768 
09:02 AAPL 96.27845
09:02 AMZN 78.42342
09:02 GOOG 75.37487


### SQL in the Q world

By now, we know what a table is and how to create one in q. Next, we'll explore what we can do with tables using qSQL, a set of SQL-like functions in q. These operations will feel familiar if you have SQL experience.

<div class="alert alert-info">
Using your previous SQL knowledge, how would you select all columns from a table?
<div>

In [14]:
select from t1

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:00 AMZN 78.44964
09:00 GOOG 75.3932 
09:01 AAPL 96.27035
09:01 AMZN 78.42006
09:01 GOOG 75.3768 
09:02 AAPL 96.27845
09:02 AMZN 78.42342
09:02 GOOG 75.37487


Filtering rows is just as simple.

In [15]:
select from t1 where sym=`AAPL

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:01 AAPL 96.27035
09:02 AAPL 96.27845


We can also create new columns or select specific ones. Here's an example of calculating the average price for `AAPL`

In [16]:
select avg price from t1 where sym=`AAPL

price   
--------
96.27031


In qSQL, `update` behaves like `select` but it creates or modifies columns without changing the original table. For example, let's add a `mean` column to our table.

In [17]:
update mean:avg price from t1

t     sym  price    mean    
----------------------------
09:00 AAPL 96.26213 83.36099
09:00 AMZN 78.44964 83.36099
09:00 GOOG 75.3932  83.36099
09:01 AAPL 96.27035 83.36099
09:01 AMZN 78.42006 83.36099
09:01 GOOG 75.3768  83.36099
09:02 AAPL 96.27845 83.36099
09:02 AMZN 78.42342 83.36099
09:02 GOOG 75.37487 83.36099


Grouping and aggregating is also straightforward using the `by` clause, similar to SQL's `GROUP BY`.

In [18]:
show kt: select avg price by sym from t1

sym | price   
----| --------
AAPL| 96.27031
AMZN| 78.43104
GOOG| 75.38162


Hmm, what's that vertical line we see in our grouped table?

This is a `keyed table`. It's actually a dictionary that maps key records to value records. We can access individual records using dictionary-like syntax.

In [19]:
kt[`AAPL]

price| 96.27031


 <div class="alert alert-warning">
 Warning! A keyed table is not a table, it's a dictionary
 <div>

In [20]:
type kt

99h


## Getting previous element

Focusing back in the problem, we aim to compute the average of the current price with the previous price for each symbol. To access the previous value in a sequence, we can use the `prev` keyword in q. Let's see how it works.

In [21]:
prev 100 200 120

0N 100 200


Notice that the first element is `0N` (null) since there's no preceding value. To avoid this, we can use the `fill` (`^`) keyword to fill the nulls with appropriate values

In [22]:
4^0N

4


In [23]:
100 200 120^100 200 120 + prev 100 200 120

100 300 320


In [24]:
(100 200 120 + prev 100 200 120)%2

0n 150 160


Let's now integrate this logic with our table operations using qSQL syntax to compute the average for each symbol's prices.

In [25]:
update price:price^%[;2]price+prev price by sym from t1

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:00 AMZN 78.44964
09:00 GOOG 75.3932 
09:01 AAPL 96.26624
09:01 AMZN 78.43485
09:01 GOOG 75.385  
09:02 AAPL 96.2744 
09:02 AMZN 78.42174
09:02 GOOG 75.37584


Finally, we just need to encapsulate it in a function

In [26]:
f : {update price:price^%[;2]price+prev price by sym from x}

We got it!

## InterConnection

kdb+ supplies interprocess communication (IPC), which enables communication between different q processes. To demonstrate this, we need to start the server side of a q process.

We can begin by listening for incoming requests by using `\p` followed by a port number.

In [27]:
\p 5010